In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

In [3]:
# Preprocess 전처리

import pandas as pd

In [6]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')
train_df.head()
test_df.head()

,name,class
0,dataset/cifar/test/0_cat.png,cat
1,dataset/cifar/test/1000_dog.png,dog
2,dataset/cifar/test/1001_airplane.png,airplane
3,dataset/cifar/test/1002_ship.png,ship
4,dataset/cifar/test/1003_deer.png,deer


In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [19]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    train_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 50000 validated image filenames belonging to 10 classes.


In [11]:
#model
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), # optimization
             loss='categorical_crossentropy', # loss function
             metrics=['accuracy']) # metrics / accuracy

In [15]:
batch_size

32

In [16]:
len(train_df) // 32

1562

In [13]:
len(train_generator)

1563

In [20]:
# training
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

/Users/sengjeawang/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1563/1563 [==============================] - 119s 76ms/step - loss: 1.7439 - accuracy: 0.3557 - val_loss: 1.3797 - val_accuracy: 0.4857
Epoch 2/10
  50/1563 [..............................] - ETA: 1:39 - loss: 1.5554 - accuracy: 0.4288

KeyboardInterrupt: 